In [23]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize
import logging
import warnings
import json
from typing import List, Tuple, Dict, Any

# Import necessary components from scikit-learn for GPR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [24]:
# --- Configuration for Balanced Refinement Phase (Week 4) ---
DATA_FILE_PATH = 'bbo_master_w04.csv' # Assumes your complete data is named this
OUTPUT_JSON_FILE = 'week04_queries.json'
X_COLUMNS = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']
DOMAIN_BOUNDS = [(0.0, 1.0) for _ in range(8)]
N_INITIAL_SAMPLES = 100 # Number of random starting points for optimisation
# XI = 0.01 provides a good balance between exploitation and exploration for this phase.
XI = 0.01 
# Output rounded to 6 decimal places
ROUNDING_PRECISION = 6

# Enforce strict [0, 1) constraint.
UPPER_BOUND_REPLACEMENT = 0.999999

# Suppress convergence warnings from GPR kernel optimization for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# Set up logging for cleaner output
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


In [25]:
def get_data_for_function(func_id: int, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, int]:
    """Filters the master dataframe for a specific function ID and extracts X and Y arrays."""
    logging.info(f"--- Processing Function F{func_id} ---")
    
    # Filter data for the specific function ID
    df_func = df[df['Function ID'] == func_id].copy()
    
    # Determine dimensionality (D) based on non-NaN X columns for the function
    X_temp = df_func[X_COLUMNS].to_numpy()
    D = np.sum(~np.isnan(X_temp[0])) if X_temp.size > 0 else 0
    
    if D == 0:
        logging.error(f"No data or dimensionality for F{func_id}. Skipping.")
        return np.array([]), np.array([]), 0
        
    # Extract X (only up to D dimensions) and Y
    X = df_func[X_COLUMNS[:D]].to_numpy()
    Y = df_func['Y'].to_numpy()
    
    logging.info(f"Data points for F{func_id}: {len(X)}. Dimensionality (D): {D}.")
    
    # CRITICAL: Get the best observed MAXIMUM (Y_max) from the entire history
    y_max = Y.max()
    logging.info(f"Current historical Y_max for F{func_id}: {y_max:.5f}")
    
    return X, Y, D

In [26]:
def expected_improvement(X_new: np.ndarray, GPR: GaussianProcessRegressor, y_max: float, xi: float) -> float:
    """
    Calculate the negative Expected Improvement (EI) for minimisation.
    We return -EI because scipy.optimize.minimize seeks to minimise the function.
    """
    X_new = X_new.reshape(1, -1)
    
    # Predict mean (mu) and standard deviation (sigma) at the new point
    mu, sigma = GPR.predict(X_new, return_std=True)
    
    mu = mu[0]
    sigma = sigma[0]
    
    if sigma == 0.0:
        return -0.0
    
    # Calculate Z score
    Z = (mu - y_max - xi) / sigma
    
    # Expected Improvement formula: E[I] = (mu - y_max - xi) * Phi(Z) + sigma * phi(Z)
    EI = (mu - y_max - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    
    # Return negative EI for minimisation
    return -EI

In [27]:
def propose_next_query(GPR: GaussianProcessRegressor, X: np.ndarray, Y: np.ndarray, D: int, xi: float) -> List[float]:
    """Finds the query point that maximises the Expected Improvement (EI)."""
    y_max = Y.max()
    bounds = DOMAIN_BOUNDS[:D]
    
    neg_ei_func = lambda x: expected_improvement(x, GPR, y_max, xi=xi)
    
    best_ei = np.inf
    best_x = None
    
    # Use multiple random restarts to find the global minimum of -EI
    for _ in range(N_INITIAL_SAMPLES):
        # Generate random initial point within bounds [0, 1]
        x0 = np.random.uniform(0.0, 1.0, size=D)
        
        # Use L-BFGS-B, a bounded minimisation algorithm
        res = minimize(neg_ei_func, x0, bounds=bounds, method='L-BFGS-B')
        
        if res.success and res.fun < best_ei:
            best_ei = res.fun
            best_x = res.x
            
    if best_x is None:
        logging.warning("Optimisation failed; picking random point as fallback.")
        return np.random.uniform(0.0, 1.0, size=D).tolist()
    
    # Ensure the result is clamped to [0.0, 1.0]
    final_x = np.clip(best_x, 0.0, 1.0).tolist()
    
    logging.info(f"Optimised Query found with EI: {-best_ei:.5f}")
    return final_x

In [28]:
def train_gpr_and_propose(func_id: int, X: np.ndarray, Y: np.ndarray, D: int, xi: float) -> List[float]:
    """Trains a GPR model and proposes the next query point for a given function."""
    if D == 0 or len(X) == 0:
        return [0.0] * D
        
    # Define the RBF Kernel with hyperparameter bounds
    kernel = C(1.0, (1e-3, 1e3)) * RBF(np.ones(D), (1e-2, 1e2))
    
    GPR = GaussianProcessRegressor(
        kernel=kernel,
        alpha=1e-6, # Small noise level (regularisation)
        n_restarts_optimizer=20, 
        normalize_y=True # Important for robust scaling
    )
    
    # Train the GPR model
    GPR.fit(X, Y)
    
    # Propose the next point by maximising EI
    next_query = propose_next_query(GPR, X, Y, D, xi)
    
    return next_query

In [29]:
def main():
    """Main function to orchestrate the Bayesian Optimisation process for all 8 functions."""
    logging.info(f"Loading master data from: {DATA_FILE_PATH}")
    try:
        df = pd.read_csv(DATA_FILE_PATH)
    except FileNotFoundError:
        logging.error(f"ERROR: Master data file not found at {DATA_FILE_PATH}. Please ensure it exists.")
        return
    
    all_queries: List[List[float]] = []
    
    # Iterate through Function IDs 1 to 8
    for func_id in range(1, 9):
        X, Y, D = get_data_for_function(func_id, df)
        
        if D == 0 or len(X) == 0:
            all_queries.append([0.0] * 8)
            continue
        
        # Train GPR and get the optimal next query point
        query_x_d = train_gpr_and_propose(func_id, X, Y, D, XI)
        
        # 1. Round the D-dimensional query
        rounded_query_x_d = [round(x, ROUNDING_PRECISION) for x in query_x_d]

        # 2. Apply the [0, 1) constraint: replace 1.0 with 0.999999
        query_x_clean = [
            UPPER_BOUND_REPLACEMENT if x >= 1.0 else x
            for x in rounded_query_x_d
        ]
        
        # Pad the cleaned query to 8 dimensions with 0.0s
        query_x_8d = query_x_clean + [0.0] * (8 - D)
        all_queries.append(query_x_8d)
        
        logging.info(f"F{func_id} Query ({D}D): {query_x_d}")
    
    # Output the final 8-dimensional queries as a JSON file
    logging.info("\n--- Final Queries JSON ---")
    logging.info(json.dumps(all_queries, indent=2))
    logging.info("----------------------------------")
    
    # Save the output to a JSON file
    with open(OUTPUT_JSON_FILE, 'w') as f:
        json.dump(all_queries, f, indent=2)
    
    logging.info(f"\nINFO: Successfully generated 8 new queries. Saved to {OUTPUT_JSON_FILE}")


In [30]:
if __name__ == "__main__":
    main()

INFO: Loading master data from: bbo_master_w04.csv
INFO: --- Processing Function F1 ---
INFO: Data points for F1: 13. Dimensionality (D): 2.
INFO: Current historical Y_max for F1: 0.06897
INFO: Optimised Query found with EI: 0.00000
INFO: F1 Query (2D): [0.5987146659727064, 0.6313764532646933]
INFO: --- Processing Function F2 ---
INFO: Data points for F2: 13. Dimensionality (D): 2.
INFO: Current historical Y_max for F2: 0.61121
INFO: Optimised Query found with EI: 0.03716
INFO: F2 Query (2D): [0.6845728421667862, 1.0]
INFO: --- Processing Function F3 ---
INFO: Data points for F3: 13. Dimensionality (D): 3.
INFO: Current historical Y_max for F3: -0.01996
INFO: Optimised Query found with EI: 0.01444
INFO: F3 Query (3D): [0.963787576903963, 3.3384862084158358e-18, 0.0]
INFO: --- Processing Function F4 ---
INFO: Data points for F4: 13. Dimensionality (D): 4.
INFO: Current historical Y_max for F4: -2.97407
INFO: Optimised Query found with EI: 0.84919
INFO: F4 Query (4D): [0.2616533363434189